In [1]:
import torch
import pickle
import dill

import sys
import os
sys.path.append(os.path.abspath('../..'))
from app.classes.transformer.sequence_to_sequence_transformer import Seq2SeqTransformer
from app.classes.transformer.decoder import Decoder
from app.classes.transformer.encoder import Encoder
from app.classes.transformer.decoder_layer import DecoderLayer
from app.classes.transformer.encoder_layer import EncoderLayer
from app.classes.transformer.multi_head_attention_layer import MultiHeadAttentionLayer
from app.classes.transformer.position_wise_feed_forward_layer import PositionwiseFeedforwardLayer

In [2]:
device = torch.device("cpu") 

SRC_LANGUAGE = 'english'
TRG_LANGUAGE = 'japanese'

In [3]:
with open('../helper/transformer/text_transform.pkl', 'rb') as f:
    text_transform = dill.load(f)

with open('../../app/models/transformer/model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('../../app/models/transformer/vocab_transform.pkl', 'rb') as f:
    vocab_transform = pickle.load(f)

In [4]:
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3

In [5]:
def perform_sample_translation():
    # Perform sample translation
    src_text = text_transform[SRC_LANGUAGE]("The weather is good today").to(device)
    trg_text = text_transform[TRG_LANGUAGE]("鼻").to(device)

    src_text = src_text.reshape(1, -1)  #because batch_size is 1
    trg_text = trg_text.reshape(1, -1)

    model.eval()
    with torch.no_grad():
        output, _ = model(src_text, trg_text) #turn off teacher forcing

    output = output.squeeze(0)
    output = output[1:]
    output_max = output.argmax(1)
    mapping = vocab_transform[TRG_LANGUAGE].get_itos()
    print("Prediction: ", "".join([mapping[i] for i in output_max]))

In [6]:
perform_sample_translation()

Prediction:  からは
